# K-layer network

We will train and test a k layer network with multiple outputs to classify images from the CIFAR-10 dataset.

In [1]:
# import functions to load batch, softmax function, compute gradient, display image for each label
# and transfer model to matlab
import functions as functions

import tensorflow.keras.utils as np_utils
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# parameter of the network
k = 3
m = [50, 50]

# Training the multi-linear classifier

## Read in the data & initialize the parameters of the network

We start by extracting the data:

In [3]:
# Reads in the data from a CIFAR-10 batch file 
# Returns the image and label data in separate files.
def LoadBatch(filename):
    data_dict = functions.load_batch(filename)
    data = data_dict[b'data']
    
    # extract the labels
    labels = data_dict[b'labels']
    
    # convert to one-hot representation
    onehot_labels = np_utils.to_categorical(labels)
    
    return data.T, onehot_labels.T, labels

Now we preprocess the raw data by normalizing it (we assume the noise is Gaussian and that the data is normally distributed):

In [4]:
def normalize_data(X, eps=1e-14):    
    return (X -  np.mean(X,axis=1)[:, np.newaxis]) / (np.std(X, axis=1)[:, np.newaxis] + eps)

We know test the code until now:

In [5]:
# dir of data
data_batch_1 = "cifar-10-batches-py/data_batch_1"
# validation data
data_batch_2 = "cifar-10-batches-py/data_batch_2"

# data is (data_dimension, data_count)
# onehot_labels is (labels_count, data_count)
data, onehot_labels, labels = LoadBatch(data_batch_1)

# we do the same for the validation set
X_validation, Y_validation, labels_validation = LoadBatch(data_batch_2)

data_dimension = len(data[:,0])
labels_count = len(onehot_labels[:,0])

# Than normalize the data according to the normal distribution
data = normalize_data(data)
X_validation = normalize_data(X_validation)

print("data mean:", np.mean(data))
print("data std:", np.std(data))

print("validation mean:", np.mean(data))
print("validation std:", np.std(data))

data mean: -5.164387436214686e-18
data std: 0.9999999999999997
validation mean: -5.164387436214686e-18
validation std: 0.9999999999999997


For $l = 1,2...k-1$:
$$ s^l = W_lx^{l-1} + b_l$$
$$x^l = max(0,s^l)$$
and then finally we have:
$$s = W_kx^{k-1} + b_k$$
$$p = softmax(s)$$
We know initialize the W's and b's of the model with each entry have Gaussian random values with zero mean and standard
deviation according to the Xavier initialization :

In [6]:
# Construct the W's
def initialize_Ws(k, m, data_dimension, labels_count):
    m = m.copy()
    
    m.insert(0, data_dimension)
    m.append(labels_count)
    Ws = []
    bs = []
    
    for i in range(0, k):
        Wi_shape = (m[i+1], m[i])
        Wi = np.random.normal(0, np.sqrt(1/m[i]), Wi_shape)
        Ws.append(Wi)
        bs.append(np.zeros(m[i+1])[:,np.newaxis])
        
    return Ws, bs    

In [7]:
Ws, bs = initialize_Ws(k, m, data_dimension, labels_count)

# Forward pass and the cost function

We write the evaluation and cost function:

In [8]:
def evaluate_classifier_BN(data, Ws, bs, mean_test, var_test, gamma, beta, test):
    """
    Arguments:
    
    data: (N, data_size) if training else (N, 1) during test
    
    test: 0 if we evaluate during training time and 1 during test time
    
    """
    xs = [data]
    s = []
    s_hat = []
    k = len(Ws)
    means = []
    var = []
    
    # forward through the k layers
    for i in range(0,k-1):
        s_i = Ws[i]@xs[i] + bs[i]
        s.append(s_i)
        
        # normalize, scale and shift
        s_hat_i, mean_i, var_i = batch_normalize_scale_shift(s_i, mean_test[i], var_test[i], gamma[i], beta[i], test)
        s_hat.append(s_hat_i)
            
        xs.append(np.maximum(0, s_hat_i))
        means.append(mean_i)
        var.append(var_i)
        
    # forward through the last layer            
    p = functions.softmax(Ws[k-1]@xs[k-1] + bs[k-1])
    
    return xs, p, s, s_hat, means, var

def batch_normalize_scale_shift(x, mean_test, var_test, gamma, beta, test):
    """
    Normalize, scale and shift the data
    _________
    Arguments:
    
    data: (N, data_size) if training else (N, 1) during test
    
    """
    # normalize
    x, mean, var = batch_normalize(x, mean_test, var_test, test)
                
    # scale and shift
    x = x * gamma[:, np.newaxis] + beta[:, np.newaxis]
    
    return x, mean, var

# normalize the data with mean and var
def batch_normalize(x, mean_test, var_test, test):
    epsilon = 1e-14

    # compute var and mean
    mean = np.mean(x, axis=1) if test == 0 else mean_test
    var = np.var(x, axis=1)  if test == 0 else var_test
    x = (x -  mean[:, np.newaxis]) / np.sqrt((var[:, np.newaxis] + epsilon))
         
    return x, mean, var

# returns the cost functions that we need to minimize
def compute_cost(data, onehot_labels, Ws, bs, lbd, gamma, beta, mean_test, var_test, test):
    cost = 0

    # p is (label_count, data_count)
    xs, p, s, s_hat, means, var = evaluate_classifier_BN(data, Ws, bs, mean_test, var_test, gamma, beta, test)
        
    # for every data in training data set
    for d in range(0,len(data[0])):
        cost -= onehot_labels[:,d].T @ np.log(p[:,d])
        
    # we devide by the data_size    
    cost /= len(data[0])
    
    # we add the regularization term
    for i in range(0,k):
        cost += lbd * np.sum(Ws[i]**2)
         
    return cost

In [9]:
gamma = [np.ones(m[i]) for i in range(k-1)]
beta = [np.zeros(m[i]) for i in range(k-1)]
compute_cost(data, onehot_labels, Ws, bs, 0, gamma, beta, np.zeros(k), np.zeros(k), 0)

2.436485411039018

We write a function that computes the accuracy of the network:

In [10]:
def compute_accuracy(data, labels, Ws, bs, gamma, beta, means, var, test):    
    # Get the index of the maximum value which is the label for each row
    xs, p, s, s_hat, means, var = evaluate_classifier_BN(data, Ws, bs, means, var, gamma, beta, test)
    predicted_labels = np.argmax(p, axis=0)
    return np.sum(labels == predicted_labels) / len(labels)

We compute the accuracy for the randomly initialized parameters. We should get an accuracy of 10% since it's random and there is 10 labels:

In [11]:
compute_accuracy(data, labels, Ws, bs, gamma, beta, np.zeros(k), np.zeros(k), 0)

0.1125

## Compute the gradients for the network parameters

We write the function that evaluates for a mini-batch the gradient of the cost function.

The mini-batch gradient is defined as follows: 

$$\textbf{W}^{t+1} = \textbf{W}^t - \eta \sum_{n \in B^t} \nabla l_{cross}(\textbf{x},\textbf{y},\textbf{W},\textbf{b})$$

In [12]:
def compute_gradient(onehot_labels, predicted_labels, Ws, xs, s, s_hat, gamma, means, var, lbd):
    data_size = len(xs[0][0])
    
    grad_Ws = []
    grad_bs = []
    
    # gradient for scale and shift
    grad_gamma = []
    grad_beta = []
            
    # g is (label_count, data_size)
    # We start by the last layer
    g = -(onehot_labels - predicted_labels)
    
    grad_Wi = (g@xs[k-1].T)/data_size + 2*lbd*Ws[k-1]
    grad_bi = np.mean(g, axis = 1)[:, np.newaxis]
    
    grad_Ws.append(grad_Wi)
    grad_bs.append(grad_bi)
        
    g = Ws[k-1].T@g
    g = g * (xs[k-1] > 0).astype(int)
    
    # now the rest of the layers
    for i in range(k-2, -1, -1):
        grad_gamma_i = np.mean(g * s_hat[i], axis=1)
        grad_beta_i = np.mean(g, axis = 1)  
        grad_gamma.append(grad_gamma_i)
        grad_beta.append(grad_beta_i)
        
        # propagate the gradients through scale and shift
        g = g*(gamma[i][:,np.newaxis])
        g = batch_norm_back_pass(g, s[i], means[i], var[i], data_size)
        
        grad_bi = np.mean(g, axis = 1)[:, np.newaxis]
        grad_Wi = (g@xs[i].T)/data_size + 2*lbd*Ws[i]
        grad_Ws.append(grad_Wi)
        grad_bs.append(grad_bi)
        
        g = Ws[i].T@g
        g = g * (xs[i] > 0).astype(int)
        
     
    grad_Ws.reverse()
    grad_bs.reverse()   
    grad_gamma.reverse()
    grad_beta.reverse()
    
    return grad_Ws, grad_bs, grad_gamma, grad_beta

def batch_norm_back_pass(g, s, mean, var, data_size):
    """
    Attributes:
    
    mean: (N, data_size)
    var: (N, data_size)
    
    """
    n_1 = np.ones((data_size,1))
    
    eps = 1e-14
    std_1 = np.power(var + eps, -0.5)[:,np.newaxis]
    std_2 = np.power(var + eps, -1.5)[:,np.newaxis]

    g1 = g * std_1
    g2 = g * std_2
        
    D = s - mean[:,np.newaxis]
    c = (g2 * D) @ n_1
    
    return g1 - ((g1 @ n_1) @ n_1.T)/data_size - (D * (c @ n_1.T))/data_size

## Train the network with cyclical learning rates

Now that we made sure the gradient descent it correct, we implement the mini batch gardient algorithm:

In [13]:
# gd_params = (s_batch, eta_min, eta_max, step_size, n_epochs)
def mini_batch_gd(data, onehot_labels, gd_params, Ws, bs, gamma, beta, lbd, X_validation, Y_validation, alpha=0.9):
    # define the parameters
    s_batch = gd_params[0]
    eta_min, eta_max, step_size = gd_params[1], gd_params[2], gd_params[3]
    n_epochs = gd_params[4]
    
    # initialize empty lists to store the loss and cost function values
    cost = []
    cost_vald = []
    accuracy = []
    accuracy_vald = []
    
    Ws_star = Ws.copy()
    bs_star = bs.copy()
    gamma_star = gamma.copy()
    beta_star = beta.copy()
    
    
    # cyclical learning rate
    step_t = 0
    l = 0
    
    # construct the mini batches
    mini_batches = construct_mini_batches(s_batch, data, onehot_labels)     
    validation_mini_batches = construct_mini_batches(s_batch, X_validation, Y_validation)
     
    for iter in range(n_epochs):
        # Shuffle the order of mini batches for each epoch
        np.random.shuffle(mini_batches)  
        for (mini_batch_X, mini_batch_y) in mini_batches:

            # compute the predictions for the mini_batch
            xs, p, s, s_hat, temp_means, temp_var = evaluate_classifier_BN(mini_batch_X, Ws_star, bs_star, np.zeros(k), np.zeros(k), gamma_star, beta_star, 0)
            
            # update means and var
            if step_t == 0:
                means = temp_means
                var = temp_var
            else:
                for i in range(0, len(means)):
                    means[i] = means[i]*alpha + temp_means[i] * (1-alpha)
                    var[i] = var[i] * alpha + temp_var[i] *(1-alpha)
            
            # update lr
            step_t, eta, l = update_cyclical_lr(step_size, l, eta_min, eta_max, step_t)
            
            # compute the new gradients
            grad_Ws, grad_bs, grad_gamma, grad_beta = compute_gradient(mini_batch_y, p, Ws_star, xs, s, s_hat, gamma_star, means, var, lbd)

            
            for i in range(0, len(Ws_star)):
                Ws_star[i] -= eta * grad_Ws[i]
                bs_star[i] -= eta * grad_bs[i]
            for i in range(0, len(gamma_star)):
                gamma_star[i] -= eta * grad_gamma[i]
                beta_star[i] -= eta * grad_beta[i]
            
            if step_t % 400 == 0:
                # compute the loss and cost function values
                cost.append(compute_cost(mini_batch_X, mini_batch_y, Ws_star, bs_star, lbd, gamma_star, beta_star, np.zeros(k), np.zeros(k), 0))
                cost_vald.append(compute_cost(X_validation, Y_validation, Ws_star, bs_star, lbd, gamma_star, beta_star, np.zeros(k), np.zeros(k), 0))

        #accuracy.append(compute_accuracy(mini_batch_X, np.argmax(mini_batch_y, axis=0), W1_star, b1_star, W2_star, b2_star))
        #accuracy_vald.append(compute_accuracy(X_validation, np.argmax(Y_validation, axis=0), W1_star, b1_star, W2_star, b2_star))
    
    plot_loss_cost(cost, cost_vald)
    #plot_accuracy(accuracy, accuracy_vald)
    return Ws_star, bs_star, gamma_star, beta_star, means, var

Auxiliary functions needed by mini_batch_gradient:

In [14]:
# return a tuple of arrays (x_batch, y_batch)
def construct_mini_batches(s_batch, data, onehot_labels):
    nb_batch = int(np.ceil(len(data[0])/s_batch))
    
    mini_batches = []
    
    for j in range(nb_batch):
        # set the start and end index of the batch
        j_start = j*s_batch
        j_end = (j+1)*s_batch        
        x_batch = data[:,j_start:j_end]
        y_batch = onehot_labels[:,j_start:j_end]
        
        mini_batches.append((x_batch, y_batch))
        
    return mini_batches

# function to update learning rate following the cyclical algorithm
def update_cyclical_lr(step_size, l, eta_min, eta_max, step_t):
    # compute the learning rate
    if 2*l*step_size <= step_t <= (2*l + 1)*step_size:
        eta = eta_min + (eta_max - eta_min) * (step_t-2*l*step_size) / step_size
    else:
        eta = eta_max - (eta_max - eta_min) * (step_t-(2*l + 1)*step_size) / step_size
        
    step_t += 1
    if step_t % (step_size*2) == 0:
        l += 1
    return step_t, eta, l

# plot the cost function values after each epoch
def plot_loss_cost(cost, cost_vald):
    step_size = 100
    x_axis = [i*step_size for i in range(len(cost))] # get the step size values as the x-axis
    plt.plot(x_axis, cost, label='Train cost')
    plt.plot(x_axis, cost_vald, label='Validation cost')
    plt.xlabel('step_size')
    plt.ylabel('cost')
    plt.legend()
    plt.show()
    
# plot accuracy
def plot_accuracy(accuracy, accuracy_vald):
    step_size = 100
    x_axis = [i*step_size for i in range(len(accuracy))] # get the step size values as the x-axis
    plt.plot(x_axis, accuracy, label='Train')
    plt.plot(x_axis, accuracy_vald, label='Validation')
    plt.xlabel('step_size')
    plt.ylabel('accuracy')
    plt.legend()
    plt.show()

# Train and test the network

We know load all the data:

In [15]:
# dir of data
data_batch_1 = "cifar-10-batches-py/data_batch_1"
data_batch_2 = "cifar-10-batches-py/data_batch_2"
data_batch_3 = "cifar-10-batches-py/data_batch_3"
data_batch_4 = "cifar-10-batches-py/data_batch_4"
data_batch_5 = "cifar-10-batches-py/data_batch_5"
test_batch = "cifar-10-batches-py/test_batch"

data_1, onehot_labels_1, labels_1 = LoadBatch(data_batch_1)
data_2, onehot_labels_2, labels_2 = LoadBatch(data_batch_2)
data_3, onehot_labels_3, labels_3 = LoadBatch(data_batch_3)
data_4, onehot_labels_4, labels_4 = LoadBatch(data_batch_4)
t_data_5, t_onehot_labels_5, t_labels_5 = LoadBatch(data_batch_5)
data_test, onehot_labels_test, labels_test = LoadBatch(test_batch)

data_5, onehot_labels_5, labels_5 = t_data_5[:,0:5000], t_onehot_labels_5[:,0:5000], t_labels_5[0:5000]
data_validation, onehot_labels_validation, labels_validation = t_data_5[:,5000:10000], t_onehot_labels_5[:,5000:10000], t_labels_5[5000:10000]

Normalize the data:

In [16]:
# merge all the data
data = np.hstack((data_1, data_2, data_3, data_4, data_5))
onehot_labels = np.hstack((onehot_labels_1, onehot_labels_2, onehot_labels_3, onehot_labels_4, onehot_labels_5))
labels = np.hstack((labels_1, labels_2, labels_3, labels_4, labels_5))

# normalize
data = normalize_data(data)
data_validation = normalize_data(data_validation)
data_test = normalize_data(data_test)

Train and test:

In [ ]:
# gd_params = (s_batch, eta_min, eta_max, step_size, n_epochs)
n_cycles = 3
s_batch = 100
step_size = 45000*5/s_batch
n_epochs = (n_cycles * 2 * step_size)/(45000/s_batch)

gd_params = (s_batch, 1e-3, 1e-1, step_size, int(n_epochs))
gamma = [np.ones(m[i]) for i in range(k-1)]
beta = [np.zeros(m[i]) for i in range(k-1)]
Ws, bs = initialize_Ws(k, m, data_dimension, labels_count)

lbd = 0.005

Ws_star, bs_star, gamma_star, beta_star, means, var = mini_batch_gd(data, onehot_labels, gd_params, Ws, bs, gamma, beta, lbd, data_validation, onehot_labels_validation)

compute_accuracy(data_validation, labels_validation, Ws_star, bs_star, gamma_star, beta_star, means, var, 1)

# Computation of optimal lambda

Compute random values of lambda:

In [ ]:
best_lbd = 0.005

# gd_params = (s_batch, eta_min, eta_max, step_size, n_epochs)
n_cycles = 3
s_batch = 100
step_size = 45000*5/s_batch
n_epochs = (n_cycles * 2 * step_size)/(45000/s_batch)
gd_params = (s_batch, 1e-5, 1e-1, step_size, int(n_epochs))
best_accuracy = 0

for i in range(8):
    # generate random lambda
    lbd = np.random.normal(best_lbd, best_lbd*0.001)
    Ws, bs = initialize_Ws(k, m, data_dimension, labels_count)
    Ws_star, bs_star, gamma_star, beta_star, means, var = mini_batch_gd(data, onehot_labels, gd_params, Ws, bs, gamma, beta, lbd, data_validation, onehot_labels_validation)
    accuracy = compute_accuracy(data_validation, labels_validation, Ws_star, bs_star, gamma_star, beta_star, means, var, 1)
    print("accuracy: ", accuracy)
    print("lambda: ", lbd)
    # Save the best lambda 
    if(accuracy > best_accuracy):
        best_accuracy = accuracy
        best_lbd = lbd

# gd_params = (s_batch, eta_min, eta_max, step_size, n_epochs)
n_cycles = 4
s_batch = 100
step_size = 45000*5/s_batch
n_epochs = (n_cycles * 2 * step_size)/(45000/s_batch)
gd_params = (s_batch, 1e-5, 1e-1, step_size, int(n_epochs))

for i in range(8):
    # generate random lambda
    lbd = np.random.normal(best_lbd, best_lbd*0.01)
    Ws, bs = initialize_Ws(k, m, data_dimension, labels_count)
    Ws_star, bs_star, gamma_star, beta_star, means, var = mini_batch_gd(data, onehot_labels, gd_params, Ws, bs, gamma, beta, lbd, data_validation, onehot_labels_validation)
    accuracy = compute_accuracy(data_validation, labels_validation, Ws_star, bs_star, gamma_star, beta_star, means, var, 1)
    print("accuracy: ", accuracy)
    print("lambda: ", lbd)
    # Save the best lambda 
    if(accuracy > best_accuracy):
        best_accuracy = accuracy
        best_lbd = lbd
        
print("best accuracy:", best_accuracy)
print("best lambda:", best_lbd)

In [ ]:
# gd_params = (s_batch, eta_min, eta_max, step_size, n_epochs)
n_cycles = 2
s_batch = 100
step_size = 45000*5/s_batch
n_epochs = (n_cycles * 2 * step_size)/(45000/s_batch)

gd_params = (s_batch, 1e-5, 1e-1, step_size, int(n_epochs))
gamma = [np.ones(m[i]) for i in range(k-1)]
beta = [np.zeros(m[i]) for i in range(k-1)]

lbd = best_lbd
Ws_star, bs_star, gamma_star, beta_star, means, var = mini_batch_gd(data, onehot_labels, gd_params, Ws, bs, gamma, beta, lbd, data_validation, onehot_labels_validation)

compute_accuracy(data_validation, labels_validation, Ws_star, bs_star, gamma_star, beta_star, means, var, 1)